In [5]:
#Q-Learning on single bot with static obstacles

import numpy as np
import random

# Define the environment
class GridWorld:
    def __init__(self, grid_size, target_pos, obstacles):
        self.grid_size = grid_size
        self.target_pos = target_pos
        self.obstacles = obstacles
        self.state = (0, 0)  # Initial position of the robot

    def reset(self):
        self.state = (0, 0)
        return self.state

    def step(self, action):
        x, y = self.state
        new_x, new_y = x, y

        # Move based on the action
        if action == 0 and x > 0:       # Move up
            new_x -= 1
        elif action == 1 and x < self.grid_size - 1:  # Move down
            new_x += 1
        elif action == 2 and y > 0:    # Move left
            new_y -= 1
        elif action == 3 and y < self.grid_size - 1:  # Move right
            new_y += 1

        # Check for obstacles
        if (new_x, new_y) not in self.obstacles:
            self.state = (new_x, new_y)

        # Calculate reward
        reward = 1 if self.state == self.target_pos else -0.1
        done = self.state == self.target_pos
        return self.state, reward, done

    def render(self):
        grid = np.zeros((self.grid_size, self.grid_size), dtype=int)
        x, y = self.state
        grid[x, y] = 1  # Robot position
        tx, ty = self.target_pos
        grid[tx, ty] = 9  # Target position

        for ox, oy in self.obstacles:
            grid[ox, oy] = -1  # Obstacles

        print(grid)

# Q-Learning agent
class QLearningAgent:
    def __init__(self, grid_size, learning_rate=0.1, discount_factor=0.99, epsilon=1.0, epsilon_decay=0.995):
        self.q_table = np.zeros((grid_size, grid_size, 4))  # Q-table with 4 actions
        self.lr = learning_rate
        self.gamma = discount_factor
        self.epsilon = epsilon
        self.epsilon_decay = epsilon_decay
        self.actions = [0, 1, 2, 3]  # [Up, Down, Left, Right]

    def choose_action(self, state):
        if random.uniform(0, 1) < self.epsilon:
            return random.choice(self.actions)  # Explore
        x, y = state
        return np.argmax(self.q_table[x, y])  # Exploit

    def update_q_value(self, state, action, reward, next_state):
        x, y = state
        nx, ny = next_state
        old_value = self.q_table[x, y, action]
        next_max = np.max(self.q_table[nx, ny])
        new_value = old_value + self.lr * (reward + self.gamma * next_max - old_value)
        self.q_table[x, y, action] = new_value

    def decay_epsilon(self):
        self.epsilon = max(0.1, self.epsilon * self.epsilon_decay)

# Train the agent
def train_agent(grid_size=5, target_pos=(4, 4), obstacles=[(2, 2), (3, 3)], episodes=500):
    env = GridWorld(grid_size, target_pos, obstacles)
    agent = QLearningAgent(grid_size)

    for episode in range(episodes):
        state = env.reset()
        total_reward = 0
        done = False

        while not done:
            action = agent.choose_action(state)
            next_state, reward, done = env.step(action)
            agent.update_q_value(state, action, reward, next_state)
            state = next_state
            total_reward += reward

        agent.decay_epsilon()

        if episode % 50 == 0:
            print(f"Episode {episode}, Total Reward: {total_reward:.2f}")

    return env, agent

# Test the trained agent
def test_agent(env, agent):
    state = env.reset()
    done = False
    steps = 0

    while not done and steps < 50:  # Limit steps to prevent infinite loops
        env.render()
        action = np.argmax(agent.q_table[state[0], state[1]])
        state, _, done = env.step(action)
        steps += 1

    env.render()
    if done:
        print("Target reached!")
    else:
        print("Failed to reach the target.")

# Main execution
obstacles = [(1, 1), (2, 2), (3, 1), (3, 3)]  # List of obstacle positions
env, agent = train_agent(grid_size=5, target_pos=(4, 4), obstacles=obstacles, episodes=500)
test_agent(env, agent)

Episode 0, Total Reward: -22.60
Episode 50, Total Reward: -2.90
Episode 100, Total Reward: -1.70
Episode 150, Total Reward: -0.10
Episode 200, Total Reward: -0.60
Episode 250, Total Reward: 0.30
Episode 300, Total Reward: 0.30
Episode 350, Total Reward: 0.10
Episode 400, Total Reward: 0.00
Episode 450, Total Reward: 0.30
[[ 1  0  0  0  0]
 [ 0 -1  0  0  0]
 [ 0  0 -1  0  0]
 [ 0 -1  0 -1  0]
 [ 0  0  0  0  9]]
[[ 0  0  0  0  0]
 [ 1 -1  0  0  0]
 [ 0  0 -1  0  0]
 [ 0 -1  0 -1  0]
 [ 0  0  0  0  9]]
[[ 0  0  0  0  0]
 [ 0 -1  0  0  0]
 [ 1  0 -1  0  0]
 [ 0 -1  0 -1  0]
 [ 0  0  0  0  9]]
[[ 0  0  0  0  0]
 [ 0 -1  0  0  0]
 [ 0  0 -1  0  0]
 [ 1 -1  0 -1  0]
 [ 0  0  0  0  9]]
[[ 0  0  0  0  0]
 [ 0 -1  0  0  0]
 [ 0  0 -1  0  0]
 [ 0 -1  0 -1  0]
 [ 1  0  0  0  9]]
[[ 0  0  0  0  0]
 [ 0 -1  0  0  0]
 [ 0  0 -1  0  0]
 [ 0 -1  0 -1  0]
 [ 0  1  0  0  9]]
[[ 0  0  0  0  0]
 [ 0 -1  0  0  0]
 [ 0  0 -1  0  0]
 [ 0 -1  0 -1  0]
 [ 0  0  1  0  9]]
[[ 0  0  0  0  0]
 [ 0 -1  0  0  0]
 [ 0

In [6]:
#Q-Learning on Single bot with dynamic obstacles

import numpy as np
import random

# Environment: GridWorld with Dynamic Obstacles
class GridWorldDynamicObstacles:
    def __init__(self, grid_size, target_pos, num_dynamic_obstacles):
        self.grid_size = grid_size
        self.target_pos = target_pos
        self.num_dynamic_obstacles = num_dynamic_obstacles
        self.state = (0, 0)  # Initial robot position
        self.dynamic_obstacles = self._generate_dynamic_obstacles()

    def _generate_dynamic_obstacles(self):
        obstacles = []
        for _ in range(self.num_dynamic_obstacles):
            # Randomly place dynamic obstacles in the grid, avoiding target and robot start
            while True:
                obstacle = (random.randint(0, self.grid_size-1), random.randint(0, self.grid_size-1))
                if obstacle != self.target_pos and obstacle != self.state and obstacle not in obstacles:
                    obstacles.append(obstacle)
                    break
        return obstacles

    def reset(self):
        self.state = (0, 0)
        self.dynamic_obstacles = self._generate_dynamic_obstacles()
        return self.state

    def move_dynamic_obstacles(self):
        # Move dynamic obstacles randomly within the grid
        new_obstacles = []
        for ox, oy in self.dynamic_obstacles:
            move_dir = random.choice([(1, 0), (-1, 0), (0, 1), (0, -1)])  # Down, Up, Right, Left
            new_ox = min(max(ox + move_dir[0], 0), self.grid_size - 1)
            new_oy = min(max(oy + move_dir[1], 0), self.grid_size - 1)
            if (new_ox, new_oy) != self.target_pos and (new_ox, new_oy) != self.state and (new_ox, new_oy) not in new_obstacles:
                new_obstacles.append((new_ox, new_oy))
            else:
                new_obstacles.append((ox, oy))  # Keep the obstacle in place if moved invalidly
        self.dynamic_obstacles = new_obstacles

    def step(self, action):
        x, y = self.state
        new_x, new_y = x, y

        # Move based on the action
        if action == 0 and x > 0:       # Move up
            new_x -= 1
        elif action == 1 and x < self.grid_size - 1:  # Move down
            new_x += 1
        elif action == 2 and y > 0:    # Move left
            new_y -= 1
        elif action == 3 and y < self.grid_size - 1:  # Move right
            new_y += 1

        # Check for collisions with dynamic obstacles
        if (new_x, new_y) not in self.dynamic_obstacles:
            self.state = (new_x, new_y)

        # Calculate reward
        reward = 1 if self.state == self.target_pos else -0.1
        done = self.state == self.target_pos
        return self.state, reward, done

    def render(self):
        grid = np.zeros((self.grid_size, self.grid_size), dtype=int)
        x, y = self.state
        grid[x, y] = 1  # Robot position
        tx, ty = self.target_pos
        grid[tx, ty] = 9  # Target position

        for ox, oy in self.dynamic_obstacles:
            grid[ox, oy] = -1  # Dynamic obstacles

        print(grid)


# QLearning Agent
class QLearningAgent:
    def __init__(self, grid_size, learning_rate=0.1, discount_factor=0.99, epsilon=1.0, epsilon_decay=0.995):
        self.q_table = np.zeros((grid_size, grid_size, 4))  # Q-table with 4 actions (Up, Down, Left, Right)
        self.lr = learning_rate
        self.gamma = discount_factor
        self.epsilon = epsilon
        self.epsilon_decay = epsilon_decay
        self.actions = [0, 1, 2, 3]  # Actions: Up, Down, Left, Right

    def choose_action(self, state):
        if random.uniform(0, 1) < self.epsilon:
            return random.choice(self.actions)  # Explore
        x, y = state
        return np.argmax(self.q_table[x, y])  # Exploit (choose best action)

    def update_q_value(self, state, action, reward, next_state):
        x, y = state
        nx, ny = next_state
        old_value = self.q_table[x, y, action]
        next_max = np.max(self.q_table[nx, ny])
        new_value = old_value + self.lr * (reward + self.gamma * next_max - old_value)
        self.q_table[x, y, action] = new_value

    def decay_epsilon(self):
        self.epsilon *= self.epsilon_decay  # Decay epsilon for exploration-exploitation trade-off


# Training the Agent
def train_agent():
    grid_size = 5
    target_pos = (4, 4)  # Target position
    num_dynamic_obstacles = 2  # Number of dynamic obstacles

    env = GridWorldDynamicObstacles(grid_size, target_pos, num_dynamic_obstacles)
    agent = QLearningAgent(grid_size)

    num_episodes = 1000
    for episode in range(num_episodes):
        state = env.reset()
        done = False
        total_reward = 0

        while not done:
            action = agent.choose_action(state)
            next_state, reward, done = env.step(action)
            agent.update_q_value(state, action, reward, next_state)
            state = next_state

            # Move dynamic obstacles every step
            env.move_dynamic_obstacles()

            total_reward += reward

        agent.decay_epsilon()

        # Print progress for every 100 episodes
        if episode % 100 == 0:
            print(f"Episode {episode}, Total Reward: {total_reward}")

    return agent, env


# Running the trained agent in the environment
def run_agent(agent, env):
    state = env.reset()
    done = False
    while not done:
        action = agent.choose_action(state)
        next_state, reward, done = env.step(action)
        state = next_state
        env.render()
        print(f"Action taken: {action}, Reward: {reward}")
        env.move_dynamic_obstacles()  # Move dynamic obstacles in each step

# Training
agent, env = train_agent()

# Testing the trained agent
run_agent(agent, env)

Episode 0, Total Reward: -5.999999999999991
Episode 100, Total Reward: 1.1102230246251565e-16
Episode 200, Total Reward: -0.19999999999999996
Episode 300, Total Reward: 0.30000000000000004
Episode 400, Total Reward: 0.20000000000000007
Episode 500, Total Reward: 0.20000000000000007
Episode 600, Total Reward: 0.30000000000000004
Episode 700, Total Reward: -0.09999999999999987
Episode 800, Total Reward: 0.30000000000000004
Episode 900, Total Reward: 0.30000000000000004
[[ 0  0  0  0  0]
 [ 1  0 -1  0  0]
 [ 0  0  0  0  0]
 [ 0  0  0  0  0]
 [-1  0  0  0  9]]
Action taken: 1, Reward: -0.1
[[ 0  0 -1  0  0]
 [ 0  0  0  0  0]
 [ 1  0  0  0  0]
 [ 0  0  0  0  0]
 [ 0 -1  0  0  9]]
Action taken: 1, Reward: -0.1
[[ 0 -1  0  0  0]
 [ 0  0  0  0  0]
 [ 0  1  0  0  0]
 [ 0  0  0  0  0]
 [ 0  0 -1  0  9]]
Action taken: 3, Reward: -0.1
[[ 0  0 -1  0  0]
 [ 0  0  0  0  0]
 [ 0  0  1  0  0]
 [ 0  0  0  0  0]
 [ 0 -1  0  0  9]]
Action taken: 3, Reward: -0.1
[[ 0 -1  0  0  0]
 [ 0  0  0  0  0]
 [ 0  0 